# Process LevSeq

Here we want to join the LevSeq output data with the function data from the LCMS.

In [ ]:
import pandas as pd
# get all the sequnence function files and see what those specific mutations did
# Also did any of the other mutations appear in the H2NOH files
import os
from copy import deepcopy
# Get them w.r.t to a mutation
from scipy.stats import mannwhitneyu
# get all the sequnence function files and see what those specific mutations did
# Also did any of the other mutations appear in the H2NOH files
import os
import numpy as np
from tqdm import tqdm
from sciutil import SciUtil

output_dir = 'output/'

lineage_df = pd.read_csv('lineage_summary.tsv', sep='\t')
lineage_df['name'] = [f'{l}_{n}' for l, n in lineage_df[['linage', 'name']].values]
# Keep track of the parents for the LevSeq data
lineage_df['AA'] = [d.replace('*', '') for d in lineage_df['AA'].values]
parent_to_name = dict(zip(lineage_df.AA, lineage_df.name))
parents = list(set(lineage_df.AA.values))

In [ ]:


# Amino acid code conversion
AA_DICT = {
    "Ala": "A",
    "Cys": "C",
    "Asp": "D",
    "Glu": "E",
    "Phe": "F",
    "Gly": "G",
    "His": "H",
    "Ile": "I",
    "Lys": "K",
    "Leu": "L",
    "Met": "M",
    "Asn": "N",
    "Pro": "P",
    "Gln": "Q",
    "Arg": "R",
    "Ser": "S",
    "Thr": "T",
    "Val": "V",
    "Trp": "W",
    "Tyr": "Y",
    "Ter": "*",
}


def normalise_calculate_stats(processed_plate_df, value_column, normalise="standard", stats_method="mannwhitneyu", parent_label="#PARENT#", normalise_method="median", parent_to_name = None):
    # if nomrliase normalize with standard normalisation
    normalised_df = pd.DataFrame()
    all_stats_df = pd.DataFrame()
    processed_plate_df[value_column] = pd.to_numeric(processed_plate_df[value_column], errors='coerce')

    for plate in set(processed_plate_df["barcode_plate"].values):
        # Normalize for each value
        plate_parent = None
        plate_campaign = None
        sub_df = processed_plate_df[processed_plate_df["barcode_plate"] == plate].copy()
        parent_values = sub_df[sub_df["amino_acid_substitutions"] == parent_label][value_column].values
        if len(parent_values) == 0:
            u.err_p(['No parent!!! skipping plate... ', plate])
            continue
        else:
            plate_parent = sub_df[sub_df["amino_acid_substitutions"] == parent_label]['aa_sequence'].values[0]
            # By default use the median
            if normalise_method == "median":
                parent_mean = np.nanmedian(parent_values)
            else:
                parent_mean = np.nanmean(parent_values)
            parent_sd = np.nanstd(parent_values)
    
            # For each plate we normalise to the parent of that plate
            sub_df[f"{value_column} plate standard norm"] = (sub_df[value_column].values - parent_mean) / parent_sd
            sub_df[f"{value_column} fold change"] = sub_df[value_column].values/ parent_mean
            sub_df[f"{value_column} parent {normalise_method}"] = parent_mean
            sub_df[f"{value_column} parent SD"] = parent_sd

            norm_value_column = f"{value_column} plate standard norm"    
            sd_cutoff = 1.5  # The number of standard deviations we want above the parent values
            # Now for all the other mutations we want to look if they are significant, first we'll look at combinations and then individually
            grouped_by_mutations = sub_df.groupby("amino_acid_substitutions")
        
            rows = []
            for mutation, grp in grouped_by_mutations:
                # Get the values and then do a ranksum test
                if mutation != parent_label:
                    vals = list(grp[norm_value_column].values)
                    U1, p = None, None
                    # Now check if there are 3 otherwise we just do > X S.D over - won't be sig anyway.
                    if len(grp) > 2:
                        # Do stats
                        U1, p = mannwhitneyu(parent_values, vals, method="exact")
                    if normalise_method == "median":
                        mean_vals = np.nanmedian(vals)
                    else:
                        mean_vals = np.nanmean(vals)
                    std_vals = np.nanstd(vals)
                    median_vals = np.nanmedian(vals)
                    sig = mean_vals > ((sd_cutoff * parent_sd) + parent_mean)
                    rows.append([value_column, mutation, len(grp), mean_vals, std_vals, median_vals, (mean_vals - parent_mean)/parent_sd, sig, U1, p, plate_campaign, plate_parent, plate])
            stats_df = pd.DataFrame(rows, columns=["value_column",  "amino_acid_substitutions", "number of wells with amino_acid substitutions", "mean",
                    "std", "median", "standard normalized to parent mean", f"greater than > {sd_cutoff} parent", "man whitney U stat",
                    "p-value", 'campaign',  'parent', 'plate'],
                )
            all_stats_df = pd.concat([all_stats_df, stats_df])
            normalised_df = pd.concat([normalised_df, sub_df])

    return all_stats_df, normalised_df


def get_dist(seq1, seq2):
    seq1 = list(seq1)
    seq2 = list(seq2)
    dist = 0
    if len(seq1) != len(seq2):
        print(len(seq1), len(seq2), 'YIKES')
    for i in range(0, len(seq1)):
        if seq1[i] != seq2[i]:
            dist += 1
    #print(dist/len(seq1))
    return dist/len(seq1)



# Process the LevSeq data and combine with function data

In [ ]:

u = SciUtil()

all_dfs = {}
all_stats_dfs = pd.DataFrame()
all_df = pd.DataFrame()

for base_dir in ['LCMS/NOPiv-linage_Done', 'LCMS/H2NOH-linage_Done',]:
    files = [f for f in os.listdir(base_dir) if '.csv' in f]
    for f in files:
        # Get all the sequence function files.
        run_name = f.replace('.csv', '')
        if os.path.exists(os.path.join(base_dir, run_name)):
            function_files = os.listdir(os.path.join(base_dir, run_name))
            
            all_function_df = pd.DataFrame()
            for function_file in function_files:
                try:
                    # Barcode
                    barcode = function_file.split('.csv')[0].split('_')[-1]
                    function_df = pd.read_csv(f'{base_dir}/{run_name}/{function_file}', header=1)
                    # Now we want to line up the barcode with the file and then join on the wells
                    function_df['function_well'] = [x.split('-')[-1] if isinstance(x, str) else None for x in function_df['Sample Vial Number'].values]
                    function_df['function_barcode_plate'] = barcode
                    u.dp([f, set(function_df['Compound Name'].values)])
                    function_df = function_df[function_df['Compound Name'].isin(['Pdt', 'pdt'])] # We only use pdt or Pdt
                    # Convert it to numeric 
                    function_df['Area'] = pd.to_numeric(function_df['Area'], errors='coerce')

                    function_df['barcode_well'] = [f'{p}_{w}' for w, p in function_df[['function_well', 'function_barcode_plate']].values]
                    function_df['filename'] = function_file
                    all_function_df = pd.concat([all_function_df, function_df])
                except:
                    print('fuction file', function_file)
            df = pd.read_csv(f'{base_dir}/{f}')
            
            df.columns = [c.lower() for c in df.columns]
            if 'barcode_plate' not in df.columns:
                df = df.rename(columns={'protein sequence': 'aa_sequence'})
                df['barcode_plate'] = df['plate'].values

            df['barcode_well'] = [f'{p}_{w}' for w, p in df[['well', 'barcode_plate']].values]
            # Join the two
            df.set_index('barcode_well', inplace=True)
            all_function_df.set_index('barcode_well', inplace=True)
            df = df.join(all_function_df, how='left')
            df.reset_index(inplace=True, drop=True)
            df['levSeq_filename'] = run_name
            df.columns = [c.replace(' ', '_') for c in df.columns]
            df = df[['barcode_plate',
                     'plate',
                     'nucleotide_mutation',
                     'amino_acid_substitutions',
                     'well', 
                     'average_mutation_frequency', 
                     'alignment_count',
                     'aa_sequence',
                     'Sample_Acq_Order_No', 'Sample_Vial_Number', 'Sample_Name',
                     'Compound_Name', 'RT_[min]', 'Area', 'function_well',
                     'function_barcode_plate', 'filename', 'levSeq_filename']]
            df = df.loc[:,~df.columns.duplicated()].copy()
            # Need to add in the fitness and the fitness_value	smiles_string
            df['fitness_value'] = df['Area'].values
            df['smiles_string'] = 'O=C(OC)C[C@H](N)C1=CC=CC=C1'
            # Get which parent it is
            min_dist = 1000
            generation_name = None
            # Will throw an error if there is no parent (but this is good because we don't want this then!)
            aa = df[df['amino_acid_substitutions'] == '#PARENT#']['aa_sequence'].values[0].replace('*', '')
            aa = aa.replace('*', '')
            # For the parent get the distance - some might have 1 or 2 aa's different... i.e. in stop codon
            for p in parents:
                parent_dist = get_dist(p, aa)
                if parent_dist < min_dist:
                    min_dist = parent_dist
                    generation_name = parent_to_name.get(p)
                    
            df['campaign_name'] = generation_name
            all_dfs[run_name] = df

            df = df.drop_duplicates(subset=['barcode_plate', 'well'])
            df = df.fillna(0)
            df.to_csv(f'{output_dir}/seqfunc/{generation_name}_{f}')
            stats_df, df = normalise_calculate_stats(df, 'Area',
                            normalise="standard",
                            stats_method="mannwhitneyu",
                            parent_label="#PARENT#",
                            normalise_method="mean",
                            parent_to_name = parent_to_name)
            stats_df['levSeq_filename'] = run_name
            all_df = pd.concat([all_df, df])
            all_stats_dfs =  pd.concat([all_stats_dfs, stats_df])
            u.warn_p(['Success!', run_name, generation_name, 'Distance between parent and generation:', min_dist, 
                     '\n # non zero fitness: ', len(df[df['fitness_value'] != 0]), '\n Size of dataset: ', len(df), 
                     '\n # unique sustitutions: ', len(set(df['amino_acid_substitutions'].values))])

all_df

# Make some summaries about the dataset

First check what campaigns are represented.

In [ ]:
# Remove any with no fitness data
all_df = all_df[~all_df['Area'].isna()]

# Remove non-0 fitnesses
all_df = all_df[all_df['fitness_value'] > 0.0]

all_df['campaign_name'].value_counts()

In [ ]:
# Next remove any values that we're not sure about i.e. low or NA in the sequenceing

nn_df = all_df[all_df['amino_acid_substitutions'] != '#N.A.#']
nn_df = nn_df[nn_df['amino_acid_substitutions'] != '#LOW#']
nn_df = nn_df[nn_df['amino_acid_substitutions'] != 'LEN']
nn_df = nn_df[nn_df['amino_acid_substitutions'] != '#PARENT#']

nn_df['amino_acid_substitutions'].value_counts()

In [ ]:
len(nn_df)

In [ ]:
nn_df = nn_df.drop_duplicates(subset=['nucleotide_mutation', 'fitness_value'])
nn_df

# Check which positions were mutated across the campaign

In [ ]:
# Now we want the actual number of positions we can do this by computing all numbers
from collections import defaultdict
positions = []
position_dict = defaultdict(list)
position_to_value = defaultdict(dict)
for subs, value in nn_df[['amino_acid_substitutions', 'Area plate standard norm']].values:
    for pos in subs.split('_'):
        # Only keep those that weren't retained
        positions.append(int(pos[1:-1]))
        position_dict[int(pos[1:-1])].append(pos[-1])
        if position_to_value[int(pos[1:-1])].get(pos[-1]):
            value = float(value)
            position_to_value[int(pos[1:-1])][pos[-1]].append(value)
        else:
            position_to_value[int(pos[1:-1])][pos[-1]] = []
            value = float(value)
            position_to_value[int(pos[1:-1])][pos[-1]].append(value)
print(len(set(positions)))

# Check whether there are too many substitutions

In [ ]:
import matplotlib.pyplot as plt
nn_df['# substitutions'] = [len(x.split('_')) for x in nn_df['amino_acid_substitutions'].values]
plt.hist(nn_df['# substitutions'].values, bins=20)

In [ ]:
many_subs = nn_df[nn_df['# substitutions'] > 10]
many_subs # Great these just look like it's from StEP which is expected

# Save these dataframes

In [ ]:
all_df.to_csv(f'{output_dir}/LevSeq_CombinedDF_pdt.csv', index=False)
nn_df.to_csv(f'{output_dir}/LevSeq_CombinedDF_pdt_filtered.csv', index=False)

# Deduplicate variants to run folding on

In [ ]:
nn_df = nn_df.sort_values(by='Area fold change', ascending=False)
len(set(nn_df['aa_sequence'].values)) # Have a look at all the unique proteins

In [ ]:
nn_df['aa_sequence'] = [a.replace('*', '') for a in nn_df['aa_sequence'].values]
nn_df['name'] = [f'{c}_{i}' for i, c in enumerate(nn_df['campaign_name'].values)]
dedup = nn_df.drop_duplicates(subset=['aa_sequence'])
for seq in dedup['aa_sequence'].values:
    if 'LEHHHHHH' not in seq:
        print(seq)

In [ ]:
dedup.to_csv(f'{output_dir}/LevSeq_unique_variants.csv', index=False)

In [ ]:
dedup

# Run folding with the substrate and intermediate

```
import sys
from enzymetk.dock_boltz_step import Boltz
from enzymetk.save_step import Save
import pandas as pd
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'

output_dir = 'boltz_26062025'
num_threads = 5
id_col = 'name'
seq_col = 'aa_sequence'
substrate_col = 'Substrate'
intermediate_col = 'Intermediate'
df = pd.read_csv('../output/LevSeq_unique_variants.csv')
df[substrate_col] = "O=C(OC)CCC1=CC=CC=C1"
df[intermediate_col] = str(r"CC1=C(/C2=C/C3=N/C(C(C)=C3CCC(O)=O)=C\C4=C(C(C=C)=C(/C=C5N=C(C(C=C)=C\5C)/C=C1\N26)N4[Fe]6=N)C)CCC(O)=O")

df << (Boltz(id_col, seq_col, substrate_col, intermediate_col, f'{output_dir}', num_threads) >> Save(f'{output_dir}filenames.pkl'))

```